In [1]:
import pandas as pd
import selenium
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [20]:
service1 = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service1)

In [21]:
url = 'https://www.yeogi.com/'
driver.get(url)

In [22]:
wait = WebDriverWait(driver, 10)

In [53]:
# 국내 여행지 접근용 xpaths [분할]
xpaths_place = []
for i in np.arange(1,5): # (1,5) / (5,9)  / (9 ~ 13) / (13 ~ 18)
    xpaths_place.append('//*[@id="__next"]/main/section[10]/article/div[' + str(i) + ']/a')

In [54]:
# 호텔 리스트 접근 xpath
xpath_select_hotels = []
for i in np.arange(1,22):
    xpath_select_hotels.append('//*[@id="__next"]/div[1]/main/section/div[2]/a[' + str(i) + ']')

In [55]:
# 버튼은 1~7(최대) 1번과 7번은 이동화살표, 1,2 번 스킵
# 호텔 리스트 페이지 이동 접근 xpath
xpath_select_button_hotels = []
for i in np.arange(3,8):
    xpath_select_button_hotels.append('//*[@id="__next"]/div[1]/main/section/div[2]/div/div/div/button[' + str(i) + ']')

In [56]:
# 호텔 세부사항 리뷰 버튼 접근 xpath
xpath_select_button_reviews = []
for i in np.arange(3,8):
    xpath_select_button_reviews.append('//*[@id="review"]/div[7]/div/div/button[' + str(i) + ']')

In [57]:
# 별점, 작성 시기 추출
def get_stars_date(stars_date_elements):
    lists = []
    star1 = 0
    star2 = 0
    for idx in range(len(stars_date_elements)):
        try:
            star1 = len(stars_date_elements[idx].find_elements(By.CLASS_NAME, 'css-7zs8ep'))
        except NoSuchElementException as e:
            print('1점도 없음')
        try:
            star2 = len(stars_date_elements[idx].find_elements(By.CLASS_NAME, 'css-1um40mz'))
        except NoSuchElementException as e:
            pass
        point = star1 + star2/2
        date = stars_date_elements[idx].text
        lists.append([point, date])
    
    return lists

In [58]:
# 호텔 리뷰 추출, 별점&작성시기 통합
def get_reviews():
    review_dict = []
    button_num=0
    breaker = False
    while(True):
        try:
            reviews = driver.find_elements(By.CLASS_NAME,'css-nyr29c')
            stars_date_elements = driver.find_elements(By.CLASS_NAME, 'css-1xow60n')
            # 데이터 수집
        except NoSuchElementException as e:
            print('호텔 리뷰 없음')
            break
            
        review_dict_five = get_stars_date(stars_date_elements)
        
        for idx in range(len(reviews)):
            review_dict_five[idx].append(reviews[idx].text)
        
        # # 수집 기간 설정 
        # for idx in range(-5,0):
        #     if review_dict_five[idx][1] == '3개월 전':
        #         del review_dict[idx:]
        #         review_dict.extend(review_dict_five)
        #         breaker = True
        #         break
                
        # if breaker:
        #     break

        # 리뷰 데이터 통합
        review_dict.extend(review_dict_five)

        if len(review_dict) >= 50:
            break
        # 페이지 이동
        try:
            button_check = driver.find_element(By.XPATH, xpath_select_button_reviews[button_num])
            button_num +=1

            # 테스트용 10개까지
            # if count > 5:
            #     # print(count)
            #     print(f'테스트 요소 끝, 최종 댓글 개수 : {count}')
            #     break
            
            if button_check.get_attribute('disabled') is not None: # 버튼 비활성화 > 페이지 종료
                # print(f'요소 끝, 최종 댓글 개수 : {count}')
                break
                
            else:
                if button_check.get_attribute('class') == 'css-dv31e9':
                    button_num = 0
                try:
                    actions = ActionChains(driver).move_to_element(driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/section/div[1]'))
                    actions.perform()
                except NoSuchElementException as e:
                    print('action_chain fail')
                button_check.click()
                driver.implicitly_wait(5)
                
        except NoSuchElementException as e:
            print('접근 오류_버튼접근 실패 : ', xpath_select_button_reviews[button_num], button_num)
            break

    return review_dict

In [59]:

# 수정
def get_hotel_data():
    # 호텔명
    try:
        hotel_name = driver.find_element(By.CLASS_NAME,'css-e3nxqe').text
    except NoSuchElementException as e:
        print('호텔명 값 없음')
   
    # print(driver.find_element(By.CLASS_NAME,'css-e3nxqe').text)

    # 서비스 및 부대시설
    try:
        hotel_service = driver.find_element(By.CLASS_NAME,'css-1vbs9oj').text.split('\n')
        # print(driver.find_element(By.CLASS_NAME,'css-1vbs9oj').text.split('\n'))
    except NoSuchElementException as e:
        hotel_service = np.nan
        print('서비스 값 없음')
        
    # 숙소 소개
    # hotel_intro = driver.find_element(By.CLASS_NAME,'css-l22fxz').text
    # print(driver.find_element(By.CLASS_NAME,'css-l22fxz').text)
    
    # 판매자 정보 (사업자 번호)
    # try:
    #     sell_user = driver.find_element(By.CLASS_NAME,'css-h9kvaf')
    # # actions = ActionChains(driver).move_to_element(sell_user)
    # # actions.perform()
    # # sell_user.click()
    # except NoSuchElementException as e:
    #     print('판매자 정보 오류')
    # sell_loc = driver.find_elements(By.CLASS_NAME,'css-138kqsc')[2].text
    # sell_num = driver.find_elements(By.CLASS_NAME,'css-138kqsc')[3].text
    # print(sell_loc, sell_num)   
    # driver.find_element(By.CLASS_NAME,'css-kytcrd').click()
    
    try:
        sell_location = driver.find_element(By.CLASS_NAME,'css-10o74wu')
    except NoSuchElementException as e:
        print('위치 값 없음')
    sell_loc = sell_location.text
    # 리뷰 순서 변경
    review_ascending = driver.find_element(By.CLASS_NAME,'css-tfi2u7')
    actions = ActionChains(driver).move_to_element(sell_location)
    actions.perform()
    review_ascending.click()  
    driver.find_element(By.CLASS_NAME,'css-1t2cu3a').click()
    
    # 리뷰 데이터 추적
    star_reviews = get_reviews()
    
    # print('크롤링 종료')

    # 구현 요구 영역
    # 호텔명 , 서비스 및 부대시설 , 호텔 위치, 리뷰정보 통합시키고 return 하기
    return hotel_name, hotel_service, sell_loc, star_reviews


In [28]:
# 해당하는 데이터프레임 사용

# 경기 제주도 충남 인천
df_one_four = pd.DataFrame(columns=['region', 'hotel_name', 'services', 'location', 'star&reviews'])

# 대구 대전 서울 경남
df_five_eigh = pd.DataFrame(columns=['region', 'hotel_name', 'services', 'location', 'star&reviews'])

# 부산 전북 울산 광주
df_eigh_ontt = pd.DataFrame(columns=['region', 'hotel_name', 'services', 'location', 'star&reviews'])

# 강원 경북 전남 충북 세종
df_onth_ontse = pd.DataFrame(columns=['region', 'hotel_name', 'services', 'location', 'star&reviews'])

In [13]:
df_one_four.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        0 non-null      object
 1   hotel_name    0 non-null      object
 2   services      0 non-null      object
 3   location      0 non-null      object
 4   star&reviews  0 non-null      object
dtypes: object(5)
memory usage: 132.0+ bytes


In [14]:
regions_list1 = ['경기', '제주도', '충남', '인천']
regions_list2 = ['대구', '대전', '서울', '경남']
regions_list3 = ['부산', '전북', '울산', '광주']
regions_list4 = ['강원', '경북', '전남', '충북', '세종']

In [ ]:
# 여기 어때 메인 페이지 기준 탐색 시작
# 지역별 링크
# 데이터 담을 방식 지정 필요함.

driver = webdriver.Chrome(service=service1)
url = 'https://www.yeogi.com/'
driver.get(url)

for i, region_path in enumerate(xpaths_place):
    # 지역 이
    time.sleep(3)
    driver.find_element(By.XPATH, region_path).click()
    driver.implicitly_wait(10)
    
    # 숙소 유형 선택 => 호텔
    driver.find_element(By.XPATH, '//*[@id="category2"]/div/div[2]/div[3]/button').click()
    driver.implicitly_wait(10)

    button_num = 0
    breaker = False

    # tmp_count = 0
    count_hotel = 0
    while(True):
        for hotel_num in np.arange(0,21):         
            # 호텔 접근
            if count_hotel >= 50:
                breaker = True
                break
            count_hotel += 1
            try: # 호텔 접근 시도
                # [맡은 부분에 따라 변수 변경]
                driver.find_element(By.XPATH, xpath_select_hotels[hotel_num]).click()
                time.sleep(2)
                driver.switch_to.window(driver.window_handles[-1])
                hotel_name, hotel_service, sell_loc, star_reviews = get_hotel_data()
                
                # 호텔 리스트로 이동
                driver.close()
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)
                
                df_one_four.loc[len(df_one_four)] = [regions_list1[i], hotel_name, hotel_service, sell_loc, star_reviews]
                
                # print('hotel 데이터 취득')
                # tmp_count += 1
                # if tmp_count > 0:
                #     print('hotel 접근 테스트 종료')
                #     breaker = True
                #     break
                
            except NoSuchElementException as e: # 현재 페이지의 호텔 검색 종료.
            # 검색 완료 후 버튼 접근
                try:
                    button_check = driver.find_element(By.XPATH, xpath_select_button_hotels[button_num])
                    button_num += 1
                
                    if button_check.get_attribute('disabled') is not None: # 버튼 비활성화 > 페이지 종료
                        breaker = True
                        break
                    
                    else: # 버튼 입력 다음페이지
                        # 버튼이 화살표일 경우 버튼 번호 초기화
                        if button_check.get_attribute('class') == 'css-dv31e9':
                            button_num = 0
                            # print('화살표 접근')
                        button_check.click()
                        driver.implicitly_wait(5)

                except NoSuchElementException as e:
                    print('접근 오류 호텔리스트 페이지 이동')
                    breaker = True
                    break
                    
        if breaker == True:
            break    
        

    # 여기 어때 메인 페이지로 이동
    print('지역이동')
    driver.back()
    driver.implicitly_wait(10)

In [51]:
driver = webdriver.Chrome(service=service1)
url = 'https://www.yeogi.com/'
driver.get(url)

In [98]:
df_one_four.to_csv('df_one_four.csv')

In [99]:
df_one_four

,region,hotel_name,services,location,star&reviews
0,경기,워커스 호텔 안산 by 아늑,"[조식포함, 와이파이, 욕실용품, 레스토랑, 금연, TV, 엘레베이터, 에어컨, 프...",경기 안산시 상록구 일동 632-10,"[[5.0, 1일 전, 주변에 아이 놀곳 칮았다가 가장 깔끔하고 조식도 서비스로 있..."
1,경기,그랜드메르 호텔 서귀포 오션,"[와이파이, 욕실용품, 금연, TV, 무료주차, 장애인편의시설]",제주특별자치도 서귀포시 하예동 460-1,"[[4.0, 1일 전, 조식까지 있어서 가성비가 좋아요~~], [3.5, 1일 전,..."
2,경기,인천호텔 에어스테이,"[미니바, 와이파이, 욕실용품, 발렛파킹, 금연, TV, 무료주차, 장애인편의시설]",인천 중구 운서동 2798-2,"[[1.0, 1개월 전, 담배냄새가 너무 심합니다], [4.5, 1개월 전, 깔끔하..."
3,충남,스파뷰 호텔,"[반려견동반, 수영장, 미니바, 사우나, 와이파이, 욕실용품, 금연, TV, 무료주...",충남 예산군 덕산면 신평리 562,"[[4.5, 1개월 전, 부모님 예약해드렸는데 너무 좋아하셨어요 깔끔하고 편의시설 ..."
4,충남,스파뷰 호텔,"[반려견동반, 수영장, 미니바, 사우나, 와이파이, 욕실용품, 금연, TV, 무료주...",충남 예산군 덕산면 신평리 562,"[[4.5, 1개월 전, 부모님 예약해드렸는데 너무 좋아하셨어요 깔끔하고 편의시설 ..."
5,충남,대천 K호텔,"[미니바, 와이파이, 욕실용품, 금연, TV, 무료주차, 편의점, 장애인편의시설]",충남 보령시 신흑동 2267-1,"[[4.5, 18일 전, 숙소가 좋아서 좋은 면접 봤네요], [4.5, 23일 전,..."
6,충남,호텔어라이브 태안 탼 한옥비치리조트,"[와이파이, 욕실용품, 금연, 에어컨, 냉장고, 객실샤워실, 무료주차, 드라이기, ...",충남 태안군 소원면 의항리 335-50,"[[4.0, 12일 전, 좋은 추억 많이 만들었어요 시설은 한옥이니 호텔과 비교는..."
7,충남,대천 버니프렌즈호텔,"[와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 프린터사용, 라운지, 냉...",충남 보령시 남곡동 811-11,"[[4.5, 6시간 전, 키즈 전용 호텔이라 주말엔 비싸지만 평일 이용하면 저렴하게..."
8,충남,도고 토비스콘도 미니엄,"[반려견동반, 와이파이, 욕실용품, 레스토랑, 금연, TV, 객실내취사, 엘레베이터...",충남 아산시 도고면 기곡리 247-7,"[[5.0, 2일 전, 예약취소 시간 지나서 아쉬웟네요 갑자기 집안사정으로 못왓어..."
9,충남,세심천 온천 호텔,"[사우나, 와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 프린터사용, 냉...",충남 예산군 삽교읍 신리 435,"[[3.5, 6시간 전, 엄마랑 둘이 첫 예산 여행. 외관과 숙소 자체는 오래되었지..."


xpath_select_hotels

try:
    hotel_service = driver.find_element(By.CLASS_NAME,'css-1vbs9oj').text.split('\n')
except NoSuchElementException:
    hotel_service = np.null
    print('서비스 값 없음')